Originally SOFA meant Sepsis-related Organ Failure Assessment, but was renamed to sequential organ failure assessment score (note that the word 'sepsis' was emitted).

Additional Code and Pseudocode to label MIMIC-III data in accordance to the requirements of Reyna et.al., who use t_sofa and t_suspicion to obtain t_sepsis.

Requirements for t_sofa:

1. Sofa increase of 2 within 24 hours

Requirements for a suspicion: 
  
1. IV-antibiotics for atleast 72 consecutive hours and blood cultures taken

2. If IV-antibiotics administered first, blood cultures must have been taken within 24 hours

3. If blood cultures taken first, IV-antibiotics must have been ordered within 72 hours

4. Time of Suspicion T_sus: Earlier timestamp of either IV-antibiotics or blood cultures under fullfillment of the stated requirements.

Requirements for Sepsis:

1. As long as t_sofa occured no more than 24 hours before or 12 hours after t_suspicion, patient is septic.

Ideally use MIMIC-Extract to label data with IV-antibiotics administered and blood cultures taken, then check if requirements met, because MIMIC-Extract gives hourly bins, so we have the timesteps that are part of the requirements. Maybe the preprocessed MIMIC-Extract Data available at https://github.com/MLforHealth/MIMIC_Extract (Pre-processed Output) contains all the necessary data.

Questions: 
1. Since IV-Antibiotics needs to be administered for atleast 72 consecutive hours for a suspicion, does that mean the earliest time of suspicion is 72 hours after entering the ICU? -> No, because t_IV is time of ordering Antibiotics, but we still need 72 hours of patient data.
2. Do we intend to use this sepsis check only to label the MIMIC-III according to the reyna et. al paper (in order to reproduce their baseline)? Or do we also intend to utilize it during prediction; because during prediction we do not have access to 72 hours of data.



In [ ]:
"""
Idea: Prerequisites: Per Patient Array with all relevant Labvalues throughout timesteps: [Patient ID][LabVal][t]
Compute Sofa with SOFASCORE (below), IV and cultures boolean with with MIMIC EXTRACT: 
Sofa -> Integer, IV -> Boolean, Cultures -> Boolean
Compute Sepsis from Sofa, IV and Cultures -> Either True=Sepsis or False=No Sepsis

"""


In [1]:
# -*- coding: utf-8 -*-
"""
Copyright 2021 shimst3r @ https://github.com/shimst3r/sofascore
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
sofascore computes the Sepsis-related Organ Failure Assessment (SOFA) score
according to Singer et al.:
    https://doi.org/10.1001%2Fjama.2016.0287
"""
"""
Prerequisites: 
sofa relevant values of one timestep -> sofa score for specific timestep
compute for each patient at every timestep
"""

from typing import NamedTuple, Optional

__version__ = "1.2.0"


class Catecholamine(NamedTuple):
    """
    Optional dosages, only cardiovascular (for SOFA-scores 2 to 4)
    """
    name: str
    dosage: float


class Condition(NamedTuple):
    """
    Computes SOFA-Score at the time where it is called. -> Monitor this score over time for sepsis check.
    Respiration - PaO2/FiO2 (mmHg): 'partial_pressure_of_oxygen: float' & 'is_mechanically_ventilated: bool' -> 'compute_score_for_respiratory_system'
    Coagulation - Platelets (× 103/μL): 'platelets_count: int' -> 'compute_score_for_coagulation'
    Liver - Bilirubin (mg/dl): 'bilirubin_level: float' -> 'compute_score_for_liver'
    Cardiovascular: 'mean_arterial_pressure: float' & 'catecholamine: Optional[Catecholamine]' -> 'compute_score_for_cardiovascular_system'
    Central nervous system - Glasgow Coma Scale score: 'glasgow_coma_scale: int' -> 'compute_score_for_nervous_system'
    Renal - Creatinine (mg/dl) or Urine output (mL/d): 'creatinine_level: float' & 'urine_output: float' -> 'compute_score_for_kidneys'



    Questions: 
    1. Does urine_output <500ml/d always mean kidney score is 3? Or is it a combination of creatinine and urine values?
    2. Where do we get Glasgow Coma Scale from? MIMIC-III
    """
    mean_arterial_pressure: float
    catecholamine: Optional[Catecholamine]
    platelets_count: int
    creatinine_level: float
    urine_output: Optional(float)
    bilirubin_level: float
    glasgow_coma_scale: int
    partial_pressure_of_oxygen: float
    is_mechanically_ventilated: bool


def compute(condition: Condition) -> int:
    cvs_score = compute_score_for_cardiovascular_system(
        mean_arterial_pressure=condition.mean_arterial_pressure,
        catecholamine=condition.catecholamine,
    )
    cg_score = compute_score_for_coagulation(platelets_count=condition.platelets_count)
    kdny_score = compute_score_for_kidneys(creatinine_level=condition.creatinine_level)
    livr_score = compute_score_for_liver(bilirubin_level=condition.bilirubin_level)
    ns_score = compute_score_for_nervous_system(
        glasgow_coma_scale=condition.glasgow_coma_scale
    )
    rs_score = compute_score_for_respiratory_system(
        partial_pressure_of_oxygen=condition.partial_pressure_of_oxygen,
        is_mechanically_ventilated=condition.is_mechanically_ventilated,
    )
    return cvs_score + cg_score + kdny_score + livr_score + ns_score + rs_score


def compute_score_for_cardiovascular_system(
    mean_arterial_pressure: float, catecholamine: Optional[Catecholamine]
) -> int:
    """
    Computes score based on mean arterial pressure or catecholamine therapy.
    """
    if catecholamine:
        if catecholamine.name == "dopamine":
            if catecholamine.dosage <= 5:
                return 2
            if catecholamine.dosage < 15:
                return 3
            return 4
        if catecholamine.name == "dobutamine":
            return 2
        if catecholamine.name in {"epinephrine", "norepinephrine"}:
            if catecholamine.dosage <= 0.1:
                return 3
            return 4
    if mean_arterial_pressure < 70:
        return 1
    return 0


def compute_score_for_coagulation(platelets_count: int) -> int:
    """
    Computes score based on platelets count (unit is number per microliter).
    """
    if platelets_count < 20_000:
        return 4
    if platelets_count < 50_000:
        return 3
    if platelets_count < 100_000:
        return 2
    if platelets_count < 150_000:
        return 1
    return 0


def compute_score_for_kidneys(creatinine_level: float, urine_output: Optional(float)) -> int:
    """
    Computes score based on Creatinine level (unit is mg/dl) and urine output (unit is mL/d).
    """
    if creatinine_level >= 5.0 or urine_output < 200:
        return 4
    if creatinine_level >= 3.5 or urine_output < 500:
        return 3
    if creatinine_level >= 2.0:
        return 2
    if creatinine_level >= 1.2:
        return 1
    return 0


def compute_score_for_liver(bilirubin_level: float) -> int:
    """
    Computes score based on Bilirubin level (unit is mg/dl).
    """
    if bilirubin_level >= 12.0:
        return 4
    if bilirubin_level >= 6.0:
        return 3
    if bilirubin_level >= 2.0:
        return 2
    if bilirubin_level >= 1.2:
        return 1
    return 0


def compute_score_for_nervous_system(glasgow_coma_scale: int) -> int:
    """
    Computes score based on Glasgow Coma Scale, see paper by Teasdale et al.:
        https://doi.org/10.1016/S0140-6736(74)91639-0
    """
    if glasgow_coma_scale < 6:
        return 4
    if glasgow_coma_scale < 10:
        return 3
    if glasgow_coma_scale < 13:
        return 2
    if glasgow_coma_scale < 15:
        return 1
    return 0


def compute_score_for_respiratory_system(
    partial_pressure_of_oxygen: float, is_mechanically_ventilated: bool
) -> int:
    """
    Computes score based on PaO2 (unit is mmHg).
    """
    if partial_pressure_of_oxygen < 100 and is_mechanically_ventilated:
        return 4
    if partial_pressure_of_oxygen < 200 and is_mechanically_ventilated:
        return 3
    if partial_pressure_of_oxygen < 300:
        return 2
    if partial_pressure_of_oxygen < 400:
        return 1
    return 0

In [5]:
"""
even though reyna et.al. compute a specific time of sepsis and we just say Sepsis=1 or No Sepsis=0, we still need to look at the whole timeseries of the patient 

Compute SepsisLabel for 1 Patient

Prerequisite:  
hourly Sofa in sofa: list, 
hourly IV administered boolean in IV_administered:list, 
hourly cultures taken boolean in cultures_taken: list, 
atleast 72 hourly bins because that is a requirement for IV check according to reyna et. al.
i am not sure yet if variables as list will work

!Testing Needed!
"""
class Sepsis(NamedTuple):
    
    sofa: list #timeseries of sofa scores
    IV_administered: list #timeseries of boolean whether IV_was administered or not
    cultures_taken: list #timeseries of boolean whether cultures were taken or not

def sepsis_check(param: Sepsis) -> int: #1 if patient is septic else 0 ToDo: Add Exceptions: if there is not t_sofa or t_IV or t_cultures or t_sus return 0 (no sepsis)
    t_sofa = get_t_sofa(patient_sofa=param.sofa)
    t_IV = iv_check(IV=param.IV_administered)
    t_cultures = blood_check(cultures=param.cultures_taken)
    t_sus = get_t_sus(IV=t_IV, cultures=t_cultures)
    return is_septic(t_sofa, t_sus)

def get_t_sofa(sofa) -> int: 
    """
    time of Sofa, probably needs to be initialized at first value, because if the first value is >2 this may already set t_sofa 
    """
    for score in sofa:
        if sofa[score] >= min(sofa[score,-24])+2: #Testing needed!! min(sofa[t,-24])+2 is supposed to be minimum of sofa from t and 24 hours back +2. Have not tested that line of code yet, so it is probably somewhat wrong.
            return score
        
def iv_check(IV) -> int: 
    """
    time of IV ordering, under the requirements of reyna et.al. Need exception if it doesnt happen -> no sepsis
    """
    consec = 0
    max = 0
    for t in IV:
        if t==False:
            consec = 0
        if t==True:
            consec += 1
            if consec > max:
                max = consec
                if max == 71:
                    return t-71 #set t_IV in the patients list? the moment where it was ordered

def blood_check(cultures) -> int: 
    """
    time of bloodcultures taken, exception needed for when it doesnt happen -> No sepsis, what if it happens more than once
    """
    for t in cultures:
        if t == True:
            return t
            
def get_t_sus(IV, cultures) -> int: 
    """
    time of suspicion, needs exception for when IV and cultures are too far apart -> No sepsis
    """
    if IV < cultures and cultures - IV <= 24:
        return IV
    if cultures < IV and IV - cultures <= 72:
        return cultures

def is_septic(sofa, sus) -> bool: 
    """
    as long as t_sofa occured no more than 24 hours before or 12 hours after t_suspicion
    """
    if sus - sofa > 24 or sofa - sus > 12: 
        return False
    else:
        return True 

In [ ]:
"""
Brainstorm, not relevant, will be deleted later

Given preprocessed Patient array containing sofa, IV, and Cultures bool [ID][LabVal][t]
Something like:
for ID in length of arrayID
    patient[has_sepsis] = sepsis.sepsis_check(sofa=array[ID][sofa][*], IV_administered=[ID][IV_Boolean][*], cultures_taken=[ID][cultures][])
"""

In [ ]:
"""
Brainstorm, not relevant, will be deleted later

def getcandidates(Patient[ID, Antibiotics, cultures, timeframe])
    for ID in patients:
        if ivcheck(patientID, AntibioticsList?/IV=True von MIMC Extract?, over each Timestep)==TRUE And bloodcheck(patientID, cultures?/cultures taken=True von MIMIC Extract, Timestep)==TRUE:
            patient is candidate #either through a candidate list, or setting a feature in patient table to 1 etc




def makecandidates(patient)
    for i in patient_id:
        if ivcheck(i)==True and bloodcheck(i)==True
            append(i) to candidatelist



"""
#Get SOFA-Candidates Pseudocode
"""
def gett_sofa(patient[condition, timestep])
    for t in patient[timestep]:
        sofa[t] = compute(patient[condition])
        if sofa[t] >= min(sofa[t,-24])+2: #min(sofa[t,-24])+2 is supposed to be minimum of sofa from t and 24 hours back +2
            patient[t_sofa] = t
        
for each patient:
    if SOFA increases by atleast 2 within 24 hours
        patient is candidate and time of sofa is the time of increasement (the timestep within a 24 hour period where the score increases to 2 above the minimum of the 24 hour period)
return candidates with time of sofa
"""


In [ ]:
"""
for i in patient_id:
    if ivcheck(i)==True and bloodcheck(i)==True
        append(i) to candidatelist

for each candidate:
    if IV-antibiotics were ordered before blood cultures were taken
        if blood cultures were taken within 24 hours afterwards
            time of suspicion is time of IV-antibiotics ordered
        else
            break
    if blood cultures were taken before IV-antibiotics were ordered
        if IV-antibiotics were ordered within 72 hours afterwards
            time of suspicion is time of blood cultures taken
        else 
            break
    return candidates with time of suspicion
"""


In [ ]:
"""
T_sepsis
Get candidates from SOFA candidates
Get candidates from suspected candidates

Merge candidates by unique identifier that occur in both candidate lists, drop everything else. -> This removes candidates where either the SOFA score did not increase or are unsuspicious given their IV-antibiotics and blood culture history.

for each remaining candidate:
    if time of sofa occured no more than 24 hours before or 12 hours after time of suspicion
        candidate is septic candidate and time of sepsis is the earlier of time of sofa and time of suspicion
return septic candidate and time of sepsis
    
Labelling MIMIC-III patients 

for each patient
    for each timestep
        if patient is septic candidate
            if timestep < time of sepsis
                the value of the sepsis-feture is 0
            else 
                the value of the sepsis-feature is 1
        else
            the value of the sepsis-feature is 0

"""